In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from catboost import CatBoostRegressor
from sklearn.decomposition import PCA
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/raw/train.csv', parse_dates=['Date'])
sample_submission = pd.read_csv('data/raw/sample_submission.csv')

In [4]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
train['Month'] = train['Date'].dt.to_period('M').astype(str)

In [5]:
monthly_agg = (
    train
    .groupby(['Company_ID', 'Product_ID', 'Month'], as_index=False)
    .agg({'Target': 'sum'})
    .rename(columns={'Target': 'sum_sales'})
    .sort_values(by=['Company_ID', 'Product_ID', 'Month'])
)

In [6]:
d = {'Company_ID': [], 'Product_ID': [], 'Month': []}
ar = monthly_agg['Company_ID'].sort_values().unique()
br = monthly_agg['Product_ID'].sort_values().unique()
cr = monthly_agg['Month'].sort_values().unique()
for company_id in ar:
    for product_id in br:
        for month in cr:
            d['Company_ID'].append(company_id)
            d['Product_ID'].append(product_id)
            d['Month'].append(month)
d = pd.DataFrame(d)
monthly_agg = d.merge(monthly_agg, 'left', on=['Company_ID', 'Product_ID', 'Month']).fillna(0)

In [7]:
monthly_agg_saved = monthly_agg.copy()
monthly_agg

,Company_ID,Product_ID,Month,sum_sales
0,0,0,2019-01,0.0
1,0,0,2019-02,0.0
2,0,0,2019-03,0.0
3,0,0,2019-04,0.0
4,0,0,2019-05,0.0
...,...,...,...,...
3459115,3,14668,2023-08,0.0
3459116,3,14668,2023-09,0.0
3459117,3,14668,2023-10,0.0
3459118,3,14668,2023-11,0.0


In [8]:
monthly_agg['Month_date'] = pd.to_datetime(monthly_agg['Month'], format='%Y-%m')

n_lags = 15

subset = []
gb = monthly_agg.groupby(['Company_ID','Product_ID'])
for i in range(n_lags + 1):
    monthly_agg[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
    subset.append(f'prev_sum_sales_{i}')
    if i > 0:
        if i + 1 <= 9:
            feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
            monthly_agg[f'rolling_sum_sales_{i + 1}'] = monthly_agg[feats_to_mean].mean(skipna=False, axis=1)
            subset.append(f'rolling_sum_sales_{i + 1}')
            monthly_agg[f'delta_regular_rolling_sum_sales_{i + 1}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'rolling_sum_sales_{i + 1}']
            subset.append(f'delta_regular_rolling_sum_sales_{i + 1}')
            if i > 1:
                monthly_agg[f'delta_rolling_sum_sales_{i}'] = monthly_agg[f'rolling_sum_sales_{i}'] - monthly_agg[f'rolling_sum_sales_{i + 1}']
                subset.append(f'delta_rolling_sum_sales_{i}')
        monthly_agg[f'delta_sum_sales_{i}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'prev_sum_sales_{i}']
        subset.append(f'delta_sum_sales_{i}')

monthly_agg['target'] = gb['sum_sales'].shift(-1)

n_components_pca = 4
pca_cols = [f'PCA_{i}' for i in range(n_components_pca)]
pca = PCA(n_components=n_components_pca)
pca.fit(monthly_agg[monthly_agg['target'] != 0][subset].fillna(monthly_agg[subset].mean()))

pca_features = pd.DataFrame(pca.transform(monthly_agg[subset].fillna(monthly_agg[subset].mean())), 
                            index=monthly_agg.index, 
                            columns=pca_cols)
monthly_agg = pd.concat([monthly_agg, pca_features], axis=1)

In [9]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales,Month_date,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_regular_rolling_sum_sales_2,delta_sum_sales_1,...,delta_sum_sales_13,prev_sum_sales_14,delta_sum_sales_14,prev_sum_sales_15,delta_sum_sales_15,target,PCA_0,PCA_1,PCA_2,PCA_3
0,0,0,2019-01,0.0,2019-01-01,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,-130.576633,8.779884,-2.307933,1.037216
1,0,0,2019-02,0.0,2019-02-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-135.785735,8.162460,0.736933,5.188205
2,0,0,2019-03,0.0,2019-03-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-140.837098,8.126269,4.127455,8.144662
3,0,0,2019-04,0.0,2019-04-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-145.782235,8.600937,7.324382,9.332052
4,0,0,2019-05,0.0,2019-05-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-150.618975,9.280941,9.742795,8.292273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459115,3,14668,2023-08,0.0,2023-08-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-180.928629,14.490193,-8.834118,1.251744
3459116,3,14668,2023-09,0.0,2023-09-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-181.244128,14.611288,-7.891511,2.994865
3459117,3,14668,2023-10,0.0,2023-10-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-181.476125,14.797797,-6.914149,4.114729
3459118,3,14668,2023-11,0.0,2023-11-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-181.671200,14.904517,-6.129591,4.381035


In [10]:
monthly_agg.dropna(subset=['target'], inplace=True)

In [11]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales,Month_date,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_regular_rolling_sum_sales_2,delta_sum_sales_1,...,delta_sum_sales_13,prev_sum_sales_14,delta_sum_sales_14,prev_sum_sales_15,delta_sum_sales_15,target,PCA_0,PCA_1,PCA_2,PCA_3
0,0,0,2019-01,0.0,2019-01-01,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,-130.576633,8.779884,-2.307933,1.037216
1,0,0,2019-02,0.0,2019-02-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-135.785735,8.162460,0.736933,5.188205
2,0,0,2019-03,0.0,2019-03-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-140.837098,8.126269,4.127455,8.144662
3,0,0,2019-04,0.0,2019-04-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-145.782235,8.600937,7.324382,9.332052
4,0,0,2019-05,0.0,2019-05-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-150.618975,9.280941,9.742795,8.292273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459114,3,14668,2023-07,0.0,2023-07-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-180.519973,14.554163,-9.535383,-0.439512
3459115,3,14668,2023-08,0.0,2023-08-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-180.928629,14.490193,-8.834118,1.251744
3459116,3,14668,2023-09,0.0,2023-09-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-181.244128,14.611288,-7.891511,2.994865
3459117,3,14668,2023-10,0.0,2023-10-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-181.476125,14.797797,-6.914149,4.114729


In [12]:
# Сортируем по Month_date (по возрастанию времени)
monthly_agg.sort_values(by=['Month_date'], inplace=True)

# Пример: берём валидацию на последние 3 месяца, которые у нас есть в данных
all_months = sorted(monthly_agg['Month_date'].unique())
N_valid_months = 3
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = monthly_agg[monthly_agg['Month_date'] < valid_threshold].sample(frac=1, random_state=42)
valid_data = monthly_agg[monthly_agg['Month_date'] >= valid_threshold]

In [13]:
feature_cols = subset# + pca_cols
cat_cols = ['Company_ID', 'Product_ID']

X_train = train_data[train_data['target'] != 0][feature_cols + cat_cols]
y_train = train_data[train_data['target'] != 0]['target']

X_valid = valid_data[valid_data['target'] != 0][feature_cols + cat_cols]
y_valid = valid_data[valid_data['target'] != 0]['target']

# Укажем, какие столбцы считать категориальными для CatBoost
# (CatBoost умеет работать с ними напрямую, выучивая таргет-кодирование)
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]  # индексы категориальных фич

In [14]:
X_train

,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_regular_rolling_sum_sales_2,delta_sum_sales_1,prev_sum_sales_2,rolling_sum_sales_3,delta_regular_rolling_sum_sales_3,delta_rolling_sum_sales_2,delta_sum_sales_2,...,prev_sum_sales_12,delta_sum_sales_12,prev_sum_sales_13,delta_sum_sales_13,prev_sum_sales_14,delta_sum_sales_14,prev_sum_sales_15,delta_sum_sales_15,Company_ID,Product_ID
29490,0.0,2.0,1.0,-1.0,-2.0,0.0,0.666667,-0.666667,0.333333,0.0,...,2.0,-2.0,6.0,-6.0,3.0,-3.0,0.0,0.0,0,499
1229635,1.0,1.0,1.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,1.0,...,0.0,1.0,2.0,-1.0,0.0,1.0,2.0,-1.0,1,6182
1560952,3.0,3.0,3.0,0.0,0.0,2.0,2.666667,0.333333,0.333333,1.0,...,0.0,3.0,0.0,3.0,0.0,3.0,2.0,1.0,1,11802
1724800,1.0,1.0,1.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,1.0,...,0.0,1.0,2.0,-1.0,1.0,0.0,0.0,1.0,1,14588
3174609,1.0,0.0,0.5,0.5,1.0,0.0,0.333333,0.666667,0.166667,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,9840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970545,3.0,2.0,2.5,0.5,1.0,1.0,2.000000,1.000000,0.500000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6375
2099764,7.0,6.0,6.5,0.5,1.0,25.0,12.666667,-5.666667,-6.166667,-18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,6274
1385092,1.0,0.0,0.5,0.5,1.0,0.0,0.333333,0.666667,0.166667,1.0,...,0.0,1.0,0.0,1.0,125.0,-124.0,2.0,-1.0,1,8828
2595552,7.0,3.0,5.0,2.0,4.0,0.0,3.333333,3.666667,1.666667,7.0,...,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3,21


In [ ]:
model = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.025,
    depth=6,
    colsample_bylevel=0.7,
    random_seed=42,
    cat_features=cat_features_indices,
    task_type='GPU',
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=1000
)

CatBoostError: catboost/private/libs/options/loss_description.cpp:20: MSE loss is not supported

In [16]:
from sklearn.metrics import mean_absolute_error
scale_factor = (monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12']
                .groupby(['Company_ID', 'Product_ID'])['sum_sales']
                .agg(lambda x: int(x.sum() > 0)).sum() * 3) / len(sample_submission)

y_pred_valid = model.predict(X_valid).round()
mae_valid = mean_absolute_error(y_valid, y_pred_valid)
print("Raw MAE (valid):    ", mae_valid)
print("Scaled MAE (valid): ", mae_valid * scale_factor)

Raw MAE (valid):     23.26182174993215
Scaled MAE (valid):  9.481987312169329


In [17]:
def predict_next_month(df, model):
    df = df.copy()
    df['Month_date'] = pd.to_datetime(df['Month'], format='%Y-%m')
    df['Month_pred'] = (df['Month_date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').astype(str)
    df = df.sort_values(by='Month_date')

    gb = df.groupby(['Company_ID', 'Product_ID'])
    for i in range(n_lags + 1):
        df[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
        if i > 0:
            if i + 1 <= 9:
                feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
                df[f'rolling_sum_sales_{i + 1}'] = df[feats_to_mean].mean(skipna=False, axis=1)
                df[f'delta_regular_rolling_sum_sales_{i + 1}'] = df['prev_sum_sales_0'] - df[f'rolling_sum_sales_{i + 1}']
                if i > 1:
                    df[f'delta_rolling_sum_sales_{i}'] = df[f'rolling_sum_sales_{i}'] - df[f'rolling_sum_sales_{i + 1}']
            df[f'delta_sum_sales_{i}'] = df['prev_sum_sales_0'] - df[f'prev_sum_sales_{i}']
    
    pca_features = pd.DataFrame(pca.transform(df[subset].fillna(df[subset].mean())), 
                                index=df.index, 
                                columns=pca_cols)
    df = pd.concat([df, pca_features], axis=1)

    df['Target'] = gb['sum_sales'].shift(-1)

    to_predict = df['Target'].isna()
    df.loc[to_predict, 'Target'] = model.predict(df.loc[to_predict, feature_cols + cat_cols]).round()
    df.loc[to_predict, 'Target'] = df.loc[to_predict, 'Target'].apply(lambda x: max(x, 0))
    return df.loc[to_predict, ['Company_ID', 'Product_ID', 'Month_pred', 'Target']]

In [18]:
mag = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') < valid_threshold].copy()
preds = []
for i in range(N_valid_months):
    p = predict_next_month(mag, model)
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    preds.append(p)
    mag = pd.concat([mag, p], axis=0)

preds = pd.concat(preds, axis=0)
valid_data = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') >= valid_threshold].copy()
merged = valid_data.merge(preds, on=['Company_ID', 'Product_ID', 'Month'], suffixes=('_actual', '_pred'))
raw_mae = mean_absolute_error(merged['sum_sales_actual'], merged['sum_sales_pred'])
print("Рекурсивный прогноз - raw MAE (valid):    ", raw_mae)
print("Рекурсивный прогноз - scaled MAE (valid): ", raw_mae * scale_factor)

Рекурсивный прогноз - raw MAE (valid):     28.63519610356319
Рекурсивный прогноз - scaled MAE (valid):  11.672282981716974


In [ ]:
full_train = monthly_agg.dropna(subset=['target']).sample(frac=1, random_state=42)
X_full = full_train[full_train['target'] != 0][feature_cols + cat_cols]
y_full = full_train[full_train['target'] != 0]['target']

final_model = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.015,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=42,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

0:	learn: 43.9081983	total: 648ms	remaining: 16m 10s
100:	learn: 28.6142205	total: 36.3s	remaining: 8m 22s
200:	learn: 23.8393344	total: 1m 7s	remaining: 7m 14s
300:	learn: 22.5560262	total: 1m 39s	remaining: 6m 38s


In [ ]:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[1]).astype(int)
sample_submission['Month_pred'] = sample_submission['Id'].apply(lambda x: x.split('_')[2]).astype(str)
sample_submission

In [20]:
preds = []
mag = monthly_agg_saved.copy()
for i in range(3):
    p = predict_next_month(mag, final_model)
    preds.append(p.copy())
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    mag = pd.concat([mag, p], axis=0)
preds = pd.concat(preds, axis=0)
sample_submission = sample_submission.drop(columns=['Target'], axis=1).merge(preds, 'left', on=['Company_ID', 'Product_ID', 'Month_pred'])

In [ ]:
to_mult = monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12'].groupby(['Company_ID', 'Product_ID'])['sum_sales'].agg(lambda x: int(x.sum() > 0)).reset_index()
for i in tqdm.tqdm(to_mult.index):
    sample_submission.loc[(sample_submission['Company_ID'] == to_mult.loc[i, 'Company_ID']) &
                          (sample_submission['Product_ID'] == to_mult.loc[i, 'Product_ID']), 'Target'] *= to_mult.loc[i, 'sum_sales']

In [ ]:
sample_submission[['Id', 'Target']]

In [23]:
sample_submission[['Id', 'Target']].to_csv('data/submissions/bruh.csv', index=False)